In [13]:
import os
import openai
import json

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
def get_day_of_week(days_ahead=0):
    import datetime
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    future_day = (datetime.datetime.now() + datetime.timedelta(days=days_ahead)).weekday()
    return days[future_day] 

In [58]:
def run_conversation(days_input):
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What day of week is it {} after today?".format(days_input)}]
    functions = [
        {
            "name": "get_day_of_week",
            "description": "Get the day of week a number of days after today",
            "parameters": {
                "type": "object",
                "properties": {
                    "days_ahead": {
                        #"type": "int",
                        "description": "The number of days after the system's current time",
                    }
                },
                "required": ["days_ahead"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
        temperature=0.0,
    )
    print(response)
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_day_of_week": get_day_of_week,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            days_ahead=function_args.get("days_ahead")
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            temperature=0.0,
        )  # get a new response from GPT where it can see the function response
        return second_response

In [59]:
response1 = run_conversation('10 days')

{
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"days_ahead\": 10\n}",
          "name": "get_day_of_week"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1687062651,
  "id": "chatcmpl-7SeLrSEJvV3lnAnpiKbGsHiZdMUKz",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 79,
    "total_tokens": 98
  }
}


In [60]:
print(response1)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "10 days after today is Tuesday.",
        "role": "assistant"
      }
    }
  ],
  "created": 1687062652,
  "id": "chatcmpl-7SeLsvmsSSgVanl2fHdhQaZAY0GqG",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 49,
    "total_tokens": 56
  }
}


In [57]:
print(response1['choices'][0]['message']['content'])

10 days after today is Tuesday.
